In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.linear_model      import LinearRegression
from sklearn.model_selection   import train_test_split
from sklearn.impute            import SimpleImputer
from sklearn.preprocessing     import FunctionTransformer
from sklearn.preprocessing     import OneHotEncoder
from sklearn.preprocessing     import OrdinalEncoder
from sklearn.compose           import ColumnTransformer
from sklearn.preprocessing     import StandardScaler
from sklearn.pipeline          import Pipeline
from sklearn.decomposition     import PCA
from sklearn.tree              import DecisionTreeRegressor
from sklearn.neighbors         import KNeighborsRegressor
from sklearn.neighbors         import KNeighborsClassifier
from sklearn.model_selection   import RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, SelectFromModel, RFE
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import RidgeCV

In [2]:
cookies = pd.read_csv('cookies.csv')
cookies_val = pd.read_csv('cookies_validate.csv')

In [3]:
cookies.drop(columns=["aesthetic appeal", "diameter"], inplace=True)
cookies_val.drop(columns=["aesthetic appeal", "diameter"], inplace=True)

In [4]:
cookies['chocolate'] = 0
cookies['raisins'] = 0
cookies['oats'] = 0
cookies['nuts'] = 0
cookies['peanut butter'] = 0

mixins = ['chocolate', 'raisins', 'oats', 'nuts', 'peanut butter']
cookies['mixins'].fillna(value = ' ', axis=0, inplace=True)
for mix in mixins:
    for i in range(len(cookies)):
        if mix in [x.strip() for x in cookies.loc[i, 'mixins'].split(',')]:
            cookies.loc[i, mix] = 1
            
cookies.drop(columns=["mixins"], inplace=True)

In [5]:
cookies_val['chocolate'] = 0
cookies_val['raisins'] = 0
cookies_val['oats'] = 0
cookies_val['nuts'] = 0
cookies_val['peanut butter'] = 0

mixins = ['chocolate', 'raisins', 'oats', 'nuts', 'peanut butter']
cookies_val['mixins'].fillna(value = ' ', axis=0, inplace=True)
for mix in mixins:
    for i in range(len(cookies_val)):
        if mix in [x.strip() for x in cookies_val.loc[i, 'mixins'].split(',')]:
            cookies_val.loc[i, mix] = 1
            
cookies_val.drop(columns=["mixins"], inplace=True)

In [6]:
X = cookies.drop(columns="quality")
y = cookies["quality"]
X_val = cookies_val.drop(columns="quality")
y_val = cookies_val["quality"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train_val, X_test_val, y_train_val, y_test_val = train_test_split(X_val, y_val, test_size=0.2, random_state=1)

In [7]:
num_cols = X_train.select_dtypes(include=["int64", "float64"]).columns
cat_cols = X_train.select_dtypes(exclude=["int64", "float64"]).columns

In [8]:
def fill_numeric_nulls(df):
    imputer = SimpleImputer(strategy="median")
    X = imputer.fit_transform(df)
    return X

numeric_nulls_imputer = FunctionTransformer(fill_numeric_nulls)

In [9]:
categ_encode_pipeline = ColumnTransformer([
    ("onehot_enc", OneHotEncoder(handle_unknown="ignore"), ['butter type'])
])

In [10]:
intermediate_pipeline = ColumnTransformer([
    ("num_nulls_imp", numeric_nulls_imputer, num_cols),
    ("cat_fill_pipe", categ_encode_pipeline, cat_cols)
])

In [11]:
process_pipeline = Pipeline([
    ("intermediate_pipe", intermediate_pipeline),
    ("standard_scal", StandardScaler())
])

In [17]:
X_val.drop(columns=['id'], inplace=True)

In [18]:
X.describe()

,sugar to flour ratio,sugar index,bake temp,chill time,calories,density,pH,grams baking soda,bake time,weight,crunch factor,chocolate,raisins,oats,nuts,peanut butter
count,5198.000000,5193.000000,5198.000000,5198.000000,5198.000000,5198.000000,5198.000000,5198.000000,5188.000000,5198.000000,5198.000000,5198.000000,5198.000000,5198.000000,5198.000000,5198.000000
mean,0.318049,5.402465,559.638322,30.390246,115.015294,0.995819,8.222020,0.530864,10.494758,14.381935,1.499367,0.675644,0.299731,0.227203,0.241631,0.013467
std,0.150036,4.668342,353.274062,17.268403,56.506171,0.062193,0.283323,0.150886,1.194584,3.023740,0.289205,0.468179,0.458184,0.419065,0.428113,0.115273
min,0.000000,0.600000,90.000000,0.000000,-99.000000,0.987110,7.720000,0.220000,8.000000,-99.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.240000,1.800000,380.000000,17.000000,76.000000,0.992300,8.110000,0.430000,9.500000,12.800000,1.250000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.310000,3.000000,470.000000,29.000000,118.000000,0.994800,8.210000,0.505000,10.300000,14.000000,1.500000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,0.390000,8.000000,640.000000,41.000000,155.000000,0.996907,8.320000,0.600000,11.300000,15.400000,1.750000,1.000000,1.000000,0.000000,0.000000,0.000000
max,3.000000,31.600000,6110.000000,146.500000,366.500000,5.000000,25.000000,2.000000,14.900000,31.800000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [19]:
X_val.describe()

,sugar to flour ratio,sugar index,bake temp,chill time,calories,density,pH,grams baking soda,bake time,weight,crunch factor,chocolate,raisins,oats,nuts,peanut butter
count,779.000000,779.000000,779.000000,779.000000,779.000000,779.000000,779.000000,779.000000,779.000000,779.000000,779.000000,779.000000,779.000000,779.000000,779.000000,779.000000
mean,0.322555,6.384531,480.423620,34.296534,129.188062,0.994293,8.196264,0.513504,10.614925,14.015404,1.491130,0.695764,0.293967,0.189987,0.154044,0.006418
std,0.120214,5.520385,225.928988,20.170179,50.679340,0.003502,0.153041,0.131211,1.234559,2.269722,0.282746,0.460379,0.455870,0.392543,0.361223,0.079909
min,0.000000,0.600000,120.000000,2.000000,8.000000,0.987460,7.790000,0.230000,8.500000,9.400000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.260000,1.900000,360.000000,21.000000,98.000000,0.991710,8.090000,0.420000,9.500000,12.600000,1.255000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.310000,4.800000,430.000000,33.000000,128.000000,0.994200,8.180000,0.500000,10.500000,13.600000,1.490000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,0.370000,10.000000,520.000000,45.000000,162.000000,0.996645,8.290000,0.580000,11.500000,14.800000,1.730000,1.000000,1.000000,0.000000,0.000000,0.000000
max,1.000000,65.800000,2140.000000,289.000000,440.000000,1.038980,8.760000,1.180000,14.200000,27.600000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000
